In [2]:
!pip install git+https://github.com/huggingface/transformers@v4.31-release

  Cloning https://github.com/huggingface/transformers (to revision v4.31-release) to /tmp/pip-req-build-bnjbv5ns
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-bnjbv5ns
  Running command git checkout -b v4.31-release --track origin/v4.31-release
  Switched to a new branch 'v4.31-release'
  Branch 'v4.31-release' set up to track remote branch 'v4.31-release' from 'origin'.
  Resolved https://github.com/huggingface/transformers to commit e51d7ac70ab8f3e69d3659226aa838308a668238
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.31.0-py3-none-any.whl size=7386692 sha256=2fde032380142c3c99a3508d0d7058038fead2b1bd749ff441519a76238014a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-zq3cx1f5/wh

In [3]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

import pandas as pd

2024-05-28 18:07:55.486469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 18:07:55.486567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 18:07:55.640296: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
max_input = 512
max_target = 256
batch_size = 3
model_checkpoints = "moussaKam/barthez-orangesum-abstract"

In [5]:
train_df = pd.read_csv('/kaggle/input/refomed/refomed_train.csv', sep='\t')
val_df = pd.read_csv('/kaggle/input/refomed/refomed_val.csv', sep='\t')


In [6]:
prompts = ['','', "Expliquez-moi, en termes simples ou paraphrase ou définition :",
"Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :",
]

In [7]:
train_df

,term,paraphrase
0,maladie de Chagas,infection parasitaire à Trypanosoma cruzi
1,corticoïde,"méthotrexate , mycophénolate mofétil , cycloph..."
2,polypathologie,plus d'une pathologie chronique
3,symptôme non spécifique,"hypotension orthostatique , troubles digestifs..."
4,crise généralisée,"tonicoclonique , atonique , voire à type d'abs..."
...,...,...
3976,thrombocytopénie,affection hématologique d'origine immune due à...
3977,manifestatios rhumatologique,arthrose
3978,sinusite aiguë,affections du nez et des sinus
3979,trouble ou handicap,tels que la paralysie cérébrale


In [8]:
all_train_prompts = []
all_train_responses = []
for t,a in zip(train_df.term.values, train_df.paraphrase.values):
    all_train_prompts.append(prompts[2] + t + ' [SEP]')
    all_train_responses.append(prompts[2] + t + ' [SEP]' + a)
    #break
index =100
modified_traindf = pd.DataFrame({'prompt':all_train_prompts[:],
             'responses':all_train_responses[:]})

all_val_prompts = []
all_val_responses = []
for t,a in zip(val_df.term.values, val_df.paraphrase.values):
    all_val_prompts.append(prompts[2] + t + ' [SEP]')
    all_val_responses.append(prompts[2] + t + ' [SEP]' + a)
    #break
    
modified_valdf = pd.DataFrame({'prompt':all_val_prompts[:],
             'responses':all_val_responses[:]})

from datasets import Dataset

traindata = Dataset.from_pandas(modified_traindf)
valdata = Dataset.from_pandas(modified_valdf)
traindata, valdata

(Dataset({
     features: ['prompt', 'responses'],
     num_rows: 3981
 }),
 Dataset({
     features: ['prompt', 'responses'],
     num_rows: 1063
 }))

In [10]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['prompt']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['responses'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [12]:
tokenize_train_data = traindata.map(preprocess_data, batched = True)
tokenize_val_data = valdata.map(preprocess_data, batched = True)

Map:   0%|          | 0/3981 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [12]:
!pip install rouge.score nltk py7zr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 30.4 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-non

In [13]:
!pip install evaluate rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 166.8 kB/s eta 0:00:00a 0:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=75ee24a8ef559f7a8b4780deb8ed8dac81f3d62a1fbd8e8d14451049a388d6b1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [14]:
import evaluate
from evaluate import load

rouge_scorer = evaluate.load('rouge')

def compute_rouge(pred):
    predictions, labels = pred
    #decode the predictions\n",
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #decode labels\n",
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #compute results\n",
    res = rouge_scorer.compute(predictions=decode_predictions, references=decode_labels)
    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)
    
    return {k: round(v, 4) for k, v in res.items()}
    


In [36]:
from datasets import load_dataset, load_metric

metric = load_metric('rouge')

def compute_rouge(pred):
    predictions, labels = pred
    #decode the predictions\n",
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #decode labels\n",
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #compute results\n",
    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
    #get
    res = {key: value.mid.fmeasure * 100 for key, value in res.items()}
    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)
    
    return {k: round(v, 4) for k, v in res.items()}

/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
!pip install huggingface_hub


!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('<HFTOKEN>')"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!rm -rf /kaggle/working/ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rm: cannot remove '/kaggle/working/': Device or resource busy


In [17]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

args = Seq2SeqTrainingArguments(
    'barthez-orange-ft', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size= 16,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=True, #available only with CUDA
    push_to_hub=True
    )

trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_train_data,
    eval_dataset=tokenize_val_data,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/amasi/barthez-orange-ft into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/531M [00:00<?, ?B/s]

Download file runs/May28_13-27-04_0493eeec829d/events.out.tfevents.1716902836.0493eeec829d.34.0: 100%|########…

Download file runs/May28_14-37-29_0493eeec829d/events.out.tfevents.1716907063.0493eeec829d.34.3: 100%|########…

Download file runs/May28_14-05-53_0493eeec829d/events.out.tfevents.1716905168.0493eeec829d.34.1: 100%|########…

...

Clean file runs/May28_14-07-06_0493eeec829d/events.out.tfevents.1716905234.0493eeec829d.34.2:  14%|#4        |…

Clean file runs/May28_14-05-53_0493eeec829d/events.out.tfevents.1716905168.0493eeec829d.34.1:  15%|#4        |…

Clean file training_args.bin:  23%|##2       | 1.00k/4.43k [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/1.06M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/531M [00:00<?, ?B/s]

In [18]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue May 28 18:21:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


You're using a BarthezTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,4.666234,0.671900,0.653500,0.671800,0.672100,20.000000
1,No log,0.693852,0.671800,0.653500,0.671800,0.672100,20.000000
2,No log,0.293922,0.671800,0.653500,0.671800,0.672100,20.000000
3,No log,0.208902,0.671900,0.653500,0.671800,0.672100,20.000000
4,No log,0.188014,0.671900,0.653500,0.671800,0.672100,20.000000
5,No log,0.179520,0.671900,0.653500,0.671800,0.672100,20.000000
6,No log,0.175183,0.671900,0.653500,0.671800,0.672100,20.000000
8,No log,0.173190,0.671900,0.653500,0.671800,0.672100,20.000000
8,No log,0.171646,0.671900,0.653600,0.671900,0.672200,20.000000
9,No log,0.170701,0.671900,0.653600,0.671900,0.672200,20.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=620, training_loss=0.8175233994760821, metrics={'train_runtime': 5563.1578, 'train_samples_per_second': 14.312, 'train_steps_per_second': 0.111, 'total_flos': 2.417771357090611e+16, 'train_loss': 0.8175233994760821, 'epoch': 19.92})

In [20]:
from huggingface_hub import Repository

# Define the repository where you want to push your model
repo_name = "amasi/barthez-orange-ft"
!cd /kaggle/working/ && git init && git remote add origin && git pull origin main
# Create a repository object
repo = Repository(local_dir="/kaggle/working/barthez-orange-ft/", clone_from=repo_name)

# Push the trained model to the repository
trainer.save_model("fine-tuned-model")
repo.push_to_hub(commit_message="Initial model upload")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Initialized empty Git repository in /kaggle/working/.git/
usage: git remote add [<options>] <name> <url>

    -f, --fetch           fetch the remote branches
    --tags                import all tags and associated objects when fetching
                          or do not fetch any tag at all (--no-tags)
    -t, --track <branch>  branch(es) to track
    -m, --master <branch>
                          master branch
    --mirror[=(push|fetch)]
                          set up remote as a mirror to push to or fetch from



/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/kaggle/working/barthez-orange-ft/ is already a clone of https://huggingface.co/amasi/barthez-orange-ft. Make sure you pull the latest changes with `repo.git_pull()`.


Upload file pytorch_model.bin:   0%|          | 1.00/531M [00:00<?, ?B/s]

Upload file runs/May28_18-12-37_04a002892927/events.out.tfevents.1716920508.04a002892927.34.0:   0%|          …

To https://huggingface.co/amasi/barthez-orange-ft
   7d2deb0..8bfc824  main -> main

To https://huggingface.co/amasi/barthez-orange-ft
   8bfc824..0142dfd  main -> main



In [44]:
#tokenize the conversation
model_inputs = tokenizer(sample,  max_length=512, padding='max_length', truncation=True)
#make prediction
raw_pred, _, _ = trainer.predict([model_inputs])
#decode the output
print(tokenizer.decode(raw_pred[0]))

</s><s> Expliquez-moi, en termes simples ou paraphrase ou définition :causes courantes</s>
